In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
# 학습데이터 (Q:사용자 발화, A:시스템 발화, label:[0:일상, 1:이별(부정), 2:사랑(긍정)])
# urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
# train_data = pd.read_csv('ChatBotData.csv')
# train_data.head()
train_data = pd.read_excel('C:/vscode/미니프로젝트/2차프로젝트_챗봇/data/TrainingData2.xlsx')
# train_data = train_data.iloc[:,:3]
train_data.head()



,Q,A,label
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,2
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,2
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,2
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,2
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,2


In [24]:
train_data.to_excel('C:/vscode/미니프로젝트/2차프로젝트_챗봇/data/TrainingData2.xlsx',index=False)

In [6]:
# 사전학습된 BERT모델 불러오기
# model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
# model = SentenceTransformer('jhgan/ko-sroberta-multitask') 이게 제일
# model = SentenceTransformer('jhgan/ko-sbert-sts')
# model = SentenceTransformer('jhgan/ko-sbert-multitask')
# model = SentenceTransformer('ddobokki/klue-roberta-base-nli-sts-ko-en') 
# 이것도 굿
model = SentenceTransformer('yngkyng/klue-roberta-base-sts-ko')

In [7]:
# Q의 문장 임베딩값 따로 저장
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [11]:
# 데이터를 tensor로 변환
import torch
embedding_data = torch.tensor(train_data['embedding'].tolist())

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_6416\2735008873.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  embedding_data = torch.tensor(train_data['embedding'].tolist())


In [10]:
# 임베딩 데이터를 pickle 파일로 저장
import pickle
with open('embedding_data.pickle', 'wb') as file:
    pickle.dump(embedding_data, file)

In [8]:
# 두 벡터에서 코사인 유사도 구하는 함수 정의
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

In [18]:
# 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 
# 챗봇 데이터의 임베딩 열(train_data['embedding'])에 저장해둔 
# 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 
# 코사인 유사도 값이 가장 높은 질문 샘플을 찾아내기.
# 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    # train_data['score'] = train_data.apply(lambda x: cos_sim(embedding_data, embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

---
여기까지 한 파일로 저장해서 import 해야될듯

In [19]:
# test
# return_answer('요즘 직장생활이 너무 편하고 좋은 것 같아')
return_answer('죽고싶지만 떡볶이는 먹고싶어')
# return_answer('취업해서 너무 신이 나!')

'건강에 해로운 건 알지만 인스턴트식품이 너무 맛있어서 계속 먹게 되시는군요.'

In [61]:
# 입력값 받아서 대답 반환하기
human = input('입력하세요')
print('사용자 입력값:', human)
return_answer(human)

사용자 입력값: 결혼한 지 삼 년 만에 아이를 낳았어. 진짜 기뻐


'간절히 바라던 아이를 출산하셔서 기쁘시군요. 지금 바라는 것이 있으신가요? '

---

> 저장된 임베딩값 파일 따로 불러오는 방법

In [1]:
import pandas as pd
from sentence_transformers import util
import torch
import numpy as np

In [27]:
# 사전학습된 BERT모델 불러오기
model2 = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
# model2 = SentenceTransformer('jhgan/ko-sroberta-multitask') 
# model2 = SentenceTransformer('jhgan/ko-sbert-sts')
# model = SentenceTransformer('jhgan/ko-sbert-multitask')
model = SentenceTransformer('ddobokki/klue-roberta-base-nli-sts-ko-en') 
# 이것도 굿

In [20]:
model # sts 내가 학습시킨 모델

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [9]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 64, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [7]:
model2
# jhgan/ko-sroberta-multitask

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [15]:
model2
# jhgan/ko-sbert-sts

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [28]:
model2
# sentence-transformers

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [25]:
df = pd.read_excel('C:/vscode/미니프로젝트/2차프로젝트_챗봇/data/TrainingData2.xlsx')
# Q의 문장 임베딩값 따로 저장
df['embedding'] = df.apply(lambda row: model.encode(row.Q), axis = 1)
df['embedding2'] = df.apply(lambda row: model2.encode(row.Q), axis = 1)

In [29]:
# Q의 문장 임베딩값 따로 저장
df['embedding2'] = df.apply(lambda row: model2.encode(row.Q), axis = 1)

In [11]:
# 두 벡터에서 코사인 유사도 구하는 함수 정의
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

def return_answer(question):
    embedding = model.encode(question)
    df['score'] = df.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return df.loc[df['score'].idxmax()]['A']

def return_answer2(question):
    embedding = model2.encode(question)
    df['score2'] = df.apply(lambda x: cos_sim(x['embedding2'], embedding), axis=1)
    return df.loc[df['score2'].idxmax()]['A']

In [30]:
# test
test = [
    '요즘 공부가 잘되어서 기분이 좋아',
    '죽고싶지만 떡볶이는 먹고싶어',
    '일하는 중인데 너무 졸려서 집에 가고싶어',
    '친구가 커피에 고추가루를 넣어먹는데 대체 왜 그러는지 정말 당황스러워.'
]

for i in test:
    print(return_answer(i))
for i in test:
    print(return_answer2(i))

공부가 잘 된다니 정말 좋으시겠어요. 특별한 이유가 있나요?
죽고 싶다는 극단적인 생각을 하시게 된 이유가 있나요? 무슨일이신지 말씀해주세요.
회사 생활이 무척 피곤하시군요.
친구와 어떤 일이 있으셨던 건가요? 저에게 알려주실 수 있나요?
공부법을 찾아 기분이 좋으신가 봐요. 공부법이 마음에 드나요?
남편과의 사별로 많이 슬프시겠어요.
아픈 몸으로 일하시느라 고생이 많으세요. 
시트에 커피를 쏟으셨다니 정말 화나시겠어요.


In [5]:
# def return_sim_question(input_sentence):
#     input_sentence = model.encode(input_sentence)
#     input_sentence = torch.tensor(input_sentence)
#     return input_sentence

# def chatbot(message):
#     message = message.strip()
#     # 입력 문장 임베딩
#     embedding_sentence = return_sim_question(message)

#     # 미리 구해진 임베딩 데이터와 현재 임베딩 데이터의 코사인 유사도 추출
#     cos_sim = util.pytorch_cos_sim(embedding_sentence,embedding_data)
#     # cos_sim = cos_sim.cpu()

#     #유사도가 가장 비슷한 질문 인덱스 반환
#     best_sim_idx = int(np.argmax(cos_sim))

#     #유사도가 가장 비슷한 질문에 해당하는 답변 제공
#     answer = df['A'][best_sim_idx]
#     return answer

In [20]:
# df = pd.read_excel('C:/vscode/미니프로젝트/2차프로젝트_챗봇/data/TrainingData2.xlsx')

# # pickle 파일에서 데이터 로드
# import pickle
# with open('embedding_data.pickle', 'rb') as file:
#     embedding_data = pickle.load(file)


In [22]:
# human = input('입력하세요')

# print('사용자 입력값:', human)
# #return_answer(human)
# chatbot(human)

사용자 입력값: 죽고싶지만 떡볶이는 먹고싶어


'죽고 싶다는 극단적인 생각을 하시게 된 이유가 있나요? 무슨일이신지 말씀해주세요.'